In [9]:
import subprocess
import os

import pandas as pd

import requests
from bs4 import BeautifulSoup

import json


pd.set_option('display.max_rows', 500)

# Data Understanding

GITHUB csv data I collected the data from the following  sites.
 1. RKI, webscrape (webscraping) https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
2. John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git
3. REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

## GITHUB csv data

GITHUB csv data
git clone/pull https://github.com/CSSEGISandData/COVID-19.git

In [10]:
git_pull = subprocess.Popen( "/usr/bin/git pull" , 
                     cwd = os.path.dirname( '../data/raw/COVID-19/' ), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

Error : b'The system cannot find the path specified.\r\n'
out : b''


In [11]:

data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)


In [12]:
pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/10/22,6/11/22,6/12/22,6/13/22,6/14/22,6/15/22,6/16/22,6/17/22,6/18/22,6/19/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,180864,180864,180864,181120,181178,181236,181465,181534,181574,181666
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,276638,276690,276731,276731,276821,276821,276821,277141,277141,277409
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,265925,265925,265927,265937,265943,265952,265964,265968,265971,265975
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,43224,43224,43224,43224,43224,43449,43449,43449,43449,43449
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,99761,99761,99761,99761,99761,99761,99761,99761,99761,99761


## Webscrapping

In [13]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [14]:
soup = BeautifulSoup(page.content, 'html.parser')

In [15]:
html_table=soup.find('table') # find the table, attention this works if one table exists

In [16]:
all_rows=html_table.find_all('tr')

In [17]:
final_data_list=[]

In [18]:
for pos,rows in enumerate(all_rows):
   
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
    final_data_list.append(col_list)
    

In [19]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [20]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,3.815.110,0,51.576,"464,5",16.269
3,Bayern,5.098.273,0,70.557,"537,0",24.329
4,Berlin,1.092.531,0,15.239,"415,9",4.639
5,Branden­burg,816.462,0,9.532,"376,6",5.711
6,Bremen,213.932,682,5.229,"768,8",786


## REST API CALLS

In [21]:

data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')


In [22]:
import json

In [23]:
json_object=json.loads(data.content) 

In [24]:
type(json_object)

dict

In [25]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [26]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])
    

In [27]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death,cases7_bl_per_100k,cases7_bl,death7_bl,cases7_bl_per_100k_txt,AdmUnitId
0,1,01,Schleswig-Holstein,Land,2910875,15,814840,1656280800000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,27992.957444,4.573731e+10,2.881496e+06,2594,850.053678,24744,0,"850,1",1
1,2,02,Hamburg,Freie und Hansestadt,1852478,6,623752,1656280800000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,33671.223086,2.089396e+09,4.188002e+05,2717,611.937092,11336,5,"611,9",2
2,3,03,Niedersachsen,Land,8003421,9,2574734,1656280800000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,32170.418125,1.299836e+11,4.008988e+06,9614,825.746890,66088,4,"825,7",3
3,4,04,Bremen,Freie Hansestadt,680130,5,213932,1656280800000,04,4132268b-54de-4327-ac1e-760e915112f1,31454.574861,1.119157e+09,3.357177e+05,786,768.823607,5229,3,"768,8",4
4,5,05,Nordrhein-Westfalen,Land,17925570,10,5646024,1656280800000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,31497.040261,8.782936e+10,2.648673e+06,25694,707.163008,126763,12,"707,2",5


In [33]:
pd_full_list.to_csv('../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series.csv',sep=';')

In [34]:
pd_full_list.shape[0]

16